In [3]:
from os import listdir
from os.path import isfile, join
import os
import cv2
import dlib
import numpy as np

# Define Image Path Here


def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=0.8, thickness=1):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)
    
detector1 = dlib.get_frontal_face_detector()

# Initialize Webcam
cap = cv2.VideoCapture('sridevi1.mp4')
#cap = cv2.VideoCapture(0)
img_size = 64
margin = 0.2
frame_count = 0

_videodir = "./dataset"

        # create dynamic name, like "D:\Current Download\Attachment82673"
_videodir = os.path.join(_videodir, 'sridevi')

        # create 'dynamic' dir, if it does not exist
if not os.path.exists(_videodir):
	os.makedirs(_videodir)

i=1;
while True:
    ret, frame = cap.read()
    frame_count += 1
    print(frame_count)
    if frame_count>40 and frame_count<60:
        file_name = _videodir + "/"+ str(frame_count)+ "_"+str(i)+".jpg"
        i+=1
        cv2.imshow("Face Detector", frame)
        cv2.imwrite(file_name, frame)
    elif cv2.waitKey(1) == 13:
        break
    else:
        continue
cap.release()
cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [12]:
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
protoPath =  "./face_detection_model/deploy.prototxt"
modelPath = "./face_detection_model/res10_300x300_ssd_iter_140000.caffemodel"

In [13]:
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

In [14]:
embedder = cv2.dnn.readNetFromTorch("openface_nn4.small2.v1.t7")



In [15]:
imagePaths = list(paths.list_images("./dataset"))
 
# initialize our lists of extracted facial embeddings and
# corresponding people names
knownEmbeddings = []
knownNames = []
 
# initialize the total number of faces processed
total = 0

In [16]:
print(imagePaths)

['./dataset/trisha/00005.png', './dataset/trisha/00003.png', './dataset/trisha/00001.png', './dataset/trisha/00002.png', './dataset/trisha/00000.png', './dataset/trisha/00004.png', './dataset/adrian/00003.jpg', './dataset/adrian/30_5.jpg', './dataset/adrian/27_2.jpg', './dataset/adrian/28_3.jpg', './dataset/adrian/00001.png', './dataset/adrian/00004.jpg', './dataset/adrian/00000.png', './dataset/adrian/26_1.jpg', './dataset/adrian/29_4.jpg', './dataset/adrian/00005.jpg', './dataset/adrian/00002.jpg', './dataset/sridevi/index3.jpeg', './dataset/sridevi/index.jpeg', './dataset/sridevi/index2.jpeg', './dataset/sridevi/index1.jpeg', './dataset/sridevi/41_1.jpg', './dataset/sridevi/index4.jpeg', './dataset/sridevi/sridevi.jpeg', './dataset/unknown/claire_dearing.jpg', './dataset/unknown/ellie_sattler.jpg', './dataset/unknown/john_hammond.jpg', './dataset/unknown/own_grady.jpg', './dataset/unknown/alan_grant.jpg', './dataset/unknown/ian_malcolm.jpg']


In [17]:
# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
	# extract the person name from the image path
	print("[INFO] processing image {}/{}".format(i + 1,
		len(imagePaths)))
	name = imagePath.split(os.path.sep)[-2]

	# load the image, resize it to have a width of 600 pixels (while
	# maintaining the aspect ratio), and then grab the image
	# dimensions
	image = cv2.imread(imagePath)
	image = imutils.resize(image, width=600)
	(h, w) = image.shape[:2]

	# construct a blob from the image
	imageBlob = cv2.dnn.blobFromImage(
		cv2.resize(image, (300, 300)), 1.0, (300, 300),
		(104.0, 177.0, 123.0), swapRB=False, crop=False)

	# apply OpenCV's deep learning-based face detector to localize
	# faces in the input image
	detector.setInput(imageBlob)
	detections = detector.forward()

	# ensure at least one face was found
	if len(detections) > 0:
		# we're making the assumption that each image has only ONE
		# face, so find the bounding box with the largest probability
		i = np.argmax(detections[0, 0, :, 2])
		confidence = detections[0, 0, i, 2]

		# ensure that the detection with the largest probability also
		# means our minimum probability test (thus helping filter out
		# weak detections)
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the face
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# extract the face ROI and grab the ROI dimensions
			face = image[startY:endY, startX:endX]
			(fH, fW) = face.shape[:2]

			# ensure the face width and height are sufficiently large
			if fW < 20 or fH < 20:
				continue

			# construct a blob for the face ROI, then pass the blob
			# through our face embedding model to obtain the 128-d
			# quantification of the face
			faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
				(96, 96), (0, 0, 0), swapRB=True, crop=False)
			embedder.setInput(faceBlob)
			vec = embedder.forward()

			# add the name of the person + corresponding face
			# embedding to their respective lists
			knownNames.append(name)
			knownEmbeddings.append(vec.flatten())
			total += 1


[INFO] processing image 1/30
[INFO] processing image 2/30
[INFO] processing image 3/30
[INFO] processing image 4/30
[INFO] processing image 5/30
[INFO] processing image 6/30
[INFO] processing image 7/30
[INFO] processing image 8/30
[INFO] processing image 9/30
[INFO] processing image 10/30
[INFO] processing image 11/30
[INFO] processing image 12/30
[INFO] processing image 13/30
[INFO] processing image 14/30
[INFO] processing image 15/30
[INFO] processing image 16/30
[INFO] processing image 17/30
[INFO] processing image 18/30
[INFO] processing image 19/30
[INFO] processing image 20/30
[INFO] processing image 21/30
[INFO] processing image 22/30
[INFO] processing image 23/30
[INFO] processing image 24/30
[INFO] processing image 25/30
[INFO] processing image 26/30
[INFO] processing image 27/30
[INFO] processing image 28/30
[INFO] processing image 29/30
[INFO] processing image 30/30


In [18]:
# dump the facial embeddings + names to disk
print("[INFO] serializing {} encodings...".format(total))
data = {"embeddings": knownEmbeddings, "names": knownNames}
f = open("./embedd", "wb")
f.write(pickle.dumps(data))
f.close()

[INFO] serializing 30 encodings...


In [19]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import argparse
import pickle

In [20]:
!pip install scikit-learn


In [21]:
# load the face embeddings
print("[INFO] loading face embeddings...")
data = pickle.loads(open("embedd", "rb").read())
 
# encode the labels
print("[INFO] encoding labels...")
le = LabelEncoder()
labels = le.fit_transform(data["names"])

[INFO] loading face embeddings...
[INFO] encoding labels...


In [22]:
# train the model used to accept the 128-d embeddings of the face and
# then produce the actual face recognition
print("[INFO] training model...")
recognizer = SVC(C=1.0, kernel="linear", probability=True)
recognizer.fit(data["embeddings"], labels)

[INFO] training model...


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [23]:
# write the actual face recognition model to disk
f = open("recognize", "wb")
f.write(pickle.dumps(recognizer))
f.close()
 
# write the label encoder to disk
f = open("lee", "wb")
f.write(pickle.dumps(le))
f.close()

In [24]:
 
# load the actual face recognition model along with the label encoder
recognizer = pickle.loads(open("recognize", "rb").read())
le = pickle.loads(open("lee", "rb").read())

In [25]:
# load the image, resize it to have a width of 600 pixels (while
# maintaining the aspect ratio), and then grab the image dimensions
image = cv2.imread("./images/trisha_adrian.jpg")
image = imutils.resize(image, width=600)
(h, w) = image.shape[:2]
 
# construct a blob from the image
imageBlob = cv2.dnn.blobFromImage(
	cv2.resize(image, (300, 300)), 1.0, (300, 300),
	(104.0, 177.0, 123.0), swapRB=False, crop=False)
 
# apply OpenCV's deep learning-based face detector to localize
# faces in the input image
detector.setInput(imageBlob)
detections = detector.forward()
# loop over the detections
for i in range(0, detections.shape[2]):
	# extract the confidence (i.e., probability) associated with the
	# prediction
	confidence = detections[0, 0, i, 2]
 
	# filter out weak detections
	if confidence > 0.5:
		# compute the (x, y)-coordinates of the bounding box for the
		# face
		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
		(startX, startY, endX, endY) = box.astype("int")
 
		# extract the face ROI
		face = image[startY:endY, startX:endX]
		(fH, fW) = face.shape[:2]
 
		# ensure the face width and height are sufficiently large
		if fW < 20 or fH < 20:
			continue


		# construct a blob for the face ROI, then pass the blob
		# through our face embedding model to obtain the 128-d
		# quantification of the face
		faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96),
			(0, 0, 0), swapRB=True, crop=False)
		embedder.setInput(faceBlob)
		vec = embedder.forward()
 
		# perform classification to recognize the face
		preds = recognizer.predict_proba(vec)[0]
		j = np.argmax(preds)
		proba = preds[j]
		name = le.classes_[j]

		# draw the bounding box of the face along with the associated
		# probability
		text = "{}: {:.2f}%".format(name, proba * 100)
		y = startY - 10 if startY - 10 > 10 else startY + 10
		cv2.rectangle(image, (startX, startY), (endX, endY),
			(0, 0, 255), 2)
		cv2.putText(image, text, (startX, y),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
 
# show the output image
cv2.imshow("Image", image)
cv2.waitKey(0)

13

In [54]:
cv2.destroyAllWindows()

In [55]:
print(text)

sridevi: 55.07%


In [56]:
print(name)

sridevi
